In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
with pd.option_context('display.max_columns', None):
    print(df.columns.tolist())

['Type', 'Udbudsform', 'Energimærke', 'Varmekilde', 'Byggeår', 'Rum', 'Bad', 'Toilet', 'Plan', 'Boligareal', 'Kælder', 'Udhus', 'Grundareal', 'Kontantpris', 'Ejerudgift', 'address', 'description', 'zip code', 'Ombygget', 'Etage', 'Depot i kælder', 'Carport', 'Garage', 'Drivhus', 'Udestue', 'Fritliggende overdækning', 'Erhverv', 'Garage/udhus', 'Købspris', 'Teknisk Pris', 'Ydelse', 'Ukendt', '(UDFASES) Bygning til erhvervsmæssig produktion vedrørende landbrug, gartneri, råstofudvinding o. lign', 'Fritliggende udestue', 'Ukendt bygning', 'Disp. rum', 'Pulterrum', 'Altan', 'Carport/Udhus ', 'Leje pr. md.', 'Depositum', 'Vindfang', 'Overdækket areal', 'Overdækning', 'Hønsehus', 'Legehus', 'Lade og garage', 'Tiloversbleven landbrugsbygning', 'Udhus (gildesal)', 'Stald til kvæg, får mv.', 'Maskinhus, garage mv.', 'Lade til foder, afgrøder mv.', 'pulterrum', 'Væksthus', 'Bygning til industri uden integreret produktionsapparat', 'Skur', 'Unnamed: 56', 'overdækket areal', 'Anneks', 'Anneks i ti

In [21]:
df = pd.read_csv('content.csv')
ml = df[['Byggeår', 'Rum', 'Bad', 'Toilet', 
         'Plan', 'Boligareal', 'Kontantpris', 'Ejerudgift']]
ml = ml.dropna()
ml.shape

(3868, 8)

In [22]:
ml.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3868 entries, 0 to 5069
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Byggeår      3868 non-null   float64
 1   Rum          3868 non-null   float64
 2   Bad          3868 non-null   float64
 3   Toilet       3868 non-null   float64
 4   Plan         3868 non-null   float64
 5   Boligareal   3868 non-null   float64
 6   Kontantpris  3868 non-null   object 
 7   Ejerudgift   3868 non-null   float64
dtypes: float64(7), object(1)
memory usage: 272.0+ KB


In [27]:
y = ml['Kontantpris'].str.replace('.', '').astype(float)
X = ml.drop(columns=['Kontantpris'])

C:\Users\marko\AppData\Local\Temp/ipykernel_13436/625537078.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  y = ml['Kontantpris'].str.replace('.', '').astype(float)


In [31]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, Lasso

perform = [] # Store performance
lambdas = np.logspace(-4, 4, 66) # Grid of lambdas

# FOR EACH LAMBDA, FIT A MODEL ON TRAINING DATA, CHECK PERFORMANCE ON VALIDATION AND STORE MSE
for lambda_ in lambdas:
    pipe_lasso = make_pipeline(PolynomialFeatures(), 
                               StandardScaler(),
                               Lasso(alpha=lambda_, random_state=161193))
    pipe_lasso.fit(X_train, y_train)
    y_pred = pipe_lasso.predict(X_val)
    perform.append(mse(y_pred, y_val))

# CRATE A SERIES WITH PERFORMANCE AND FIND PARM WITH LOWEST MSE    
hyperparam_perform = pd.Series(perform,index=lambdas)
optimal = hyperparam_perform.nsmallest(1)    
print('Optimal lambda:', optimal.index[0])
print('Validation MSE: %.3f' % optimal.values[0])

C:\Users\marko\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2351095290760818.5, tolerance: 770140539118.303
  model = cd_fast.enet_coordinate_descent(
C:\Users\marko\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2351095286829601.5, tolerance: 770140539118.303
  model = cd_fast.enet_coordinate_descent(
C:\Users\marko\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2351095281610414.0, tolerance: 770140539118.303
  model = cd_fast.enet_coordinate_descent(
C:\Users\marko\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarnin

Optimal lambda: 1375.5029890088583
Validation MSE: 3393994048323.273


C:\Users\marko\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1700607402142876.0, tolerance: 770140539118.303
  model = cd_fast.enet_coordinate_descent(
C:\Users\marko\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1456152728692485.0, tolerance: 770140539118.303
  model = cd_fast.enet_coordinate_descent(
C:\Users\marko\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1116986641933827.8, tolerance: 770140539118.303
  model = cd_fast.enet_coordinate_descent(
C:\Users\marko\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarnin